# Future Strategy - Factor Strategy 0002

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2025-07-31'
ed = '2026-02-13'

In [3]:
sql_factor = """
SELECT
    date,
    instrument,
    ((m_corr(m_avg(volume, 20), low, 5) + ((high + low) / 2)) - close) AS factor
FROM cn_future_bar1d
"""

In [4]:
number_security_long  = 10
number_security_short = 10  
position_allocation = "equal"
rebalance_period    = "month"
    
map_position_long = {
    "equal":"1/SUM(1)",
    "by_score":"score_long/SUM(score_long)",
    "by_score_rank":"(1/score_rank_long)/SUM(1/score_rank_long)",
}
str_position_long = map_position_long[position_allocation]

map_position_short = {
    "equal":"1/SUM(1)",
    "by_score":"score_short/SUM(score_short)",
    "by_score_rank":"(1/score_rank_short)/SUM(1/score_rank_short)",
}
str_position_short = map_position_short[position_allocation]

map_rebalance = {
    "day":"1=1",
    "week":"is_week_end_trade = 1",
    "month":"is_month_end_trade = 1",
    "quarter":"is_quarter_end_trade = 1",
    "year":"is_year_end_trade = 1",
}
str_rebalance = map_rebalance[rebalance_period]

sql_merge = f"""
WITH
data_alpha AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        1  * factor AS score_long,
        -1 * factor AS score_short,
        c_rank(-1 * factor) AS score_rank_long,
        c_rank( 1 * factor) AS score_rank_short,
        IF(score_rank_long  <= {number_security_long},  1, 0) AS trade_long,
        IF(score_rank_short <= {number_security_short}, 1, 0) AS trade_short,
    FROM data_alpha
    QUALIFY trade_long = 1 OR trade_short = 1
),
data_date AS (
    SELECT
        date,
        instrument,
        score_long, 
        score_short, 
        score_rank_long, 
        score_rank_short,
        trade_long,
        trade_short,
        IF(trade_long  = 1, {str_position_long}  OVER (PARTITION BY date, trade_long),  0) AS position_long,
        IF(trade_short = 1, {str_position_short} OVER (PARTITION BY date, trade_short), 0) AS position_short,
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE {str_rebalance}
)
SELECT *
FROM data_date
ORDER BY date, score_long
"""

In [5]:
df = dai.query(sql_merge, filters={"date":[sd, ed]}).df()
df

,date,instrument,score_long,score_short,score_rank_long,score_rank_short,trade_long,trade_short,position_long,position_short
0,2025-09-30,wr0000.SHF,-822.753779,822.753779,738.0,1.0,0,1,0.0,0.1
1,2025-09-30,ps2512.GFE,-602.429345,602.429345,737.0,2.0,0,1,0.0,0.1
2,2025-09-30,ps2603.GFE,-594.604305,594.604305,736.0,3.0,0,1,0.0,0.1
3,2025-09-30,ps2601.GFE,-549.943179,549.943179,735.0,4.0,0,1,0.0,0.1
4,2025-09-30,br2510.SHF,-541.663885,541.663885,734.0,5.0,0,1,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...
94,2026-01-30,sn0000.SHF,25466.511043,-25466.511043,5.0,801.0,1,0,0.1,0.0
95,2026-01-30,sn2602.SHF,25570.306256,-25570.306256,4.0,802.0,1,0,0.1,0.0
96,2026-01-30,sn8888.SHF,25974.061524,-25974.061524,3.0,803.0,1,0,0.1,0.0
97,2026-01-30,sn2603.SHF,25974.608767,-25974.608767,2.0,804.0,1,0,0.1,0.0


## 3. Backtest Strategy

In [6]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.data_date = dai.query("SELECT date FROM mldt_cn_stock_calendar_daily WHERE is_month_end_trade = 1").df()

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]
    if len(df_now) == 0:
        return

    from bigtrader.constant import OrderType
    from bigtrader.constant import Direction

    holding_instruments = list(context.get_account_positions().keys())

    for ins in holding_instruments:
        position_long  = context.get_position(ins, Direction.LONG)
        position_short = context.get_position(ins, Direction.SHORT)
        price = data.current(ins,"open")
        if (position_long.current_qty  != 0):
            context.sell_close(ins, position_long.avail_qty, price, order_type=OrderType.MARKET)
        if (position_short.current_qty != 0):
            context.buy_close(ins, position_short.avail_qty, price, order_type=OrderType.MARKET)

    df_now_long  = df_now[df_now['trade_long']  == 1]
    df_now_short = df_now[df_now['trade_short'] == 1]

    cash = context.portfolio.cash

    for i, x in df_now_long.iterrows():
        ins = x.instrument
        price = data.current(ins, "open")
        position = 0.0 if pd.isnull(x.position_long)  else float(x.position_long)  / 2
        volume = 1
        # volume = max(int(cash * position / price), 0) 
        context.buy_open(ins, volume, price, order_type=OrderType.MARKET)

    for i, x in df_now_short.iterrows():
        ins = x.instrument
        price = data.current(ins, "open")
        position = 0.0 if pd.isnull(x.position_short) else float(x.position_short) / 2
        volume = 1
        # volume = max(int(cash * position / price), 0)
        context.sell_open(ins, volume, price, order_type=OrderType.MARKET)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000 + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""期货""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""open""",
    order_price_field_sell="""open""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-27 10:55:46] [info     ] bigtrader.v34 开始运行 ..
[2026-02-27 10:55:46] [info     ] 2025-09-30, 2026-01-30, , future, instruments=60
[2026-02-27 10:55:46] [info     ] bigtrader module V2.2.0
[2026-02-27 10:55:46] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10


strategy(bktfut): order() not find contract for symbol:br2510.SHF


2026-02-27 10:55:46.775930 strategy(bktfut): order() not find contract for symbol:br2510.SHF 
2026-02-27 10:55:46.776610 strategy(bktfut): order() not find contract for symbol:br2510.SHF 


strategy(bktfut): order() not find contract for symbol:CJ605.CZC


2026-02-27 10:55:46.914701 strategy(bktfut): order() not find contract for symbol:CJ605.CZC 
2026-02-27 10:55:46.915377 strategy(bktfut): order() not find contract for symbol:CJ605.CZC 


...

strategy(bktfut): order() not find contract for symbol:PX602.CZC


2026-02-27 10:55:47.074106 strategy(bktfut): order() not find contract for symbol:PX602.CZC 
2026-02-27 10:55:47.074804 strategy(bktfut): order() not find contract for symbol:PX602.CZC 


strategy(bktfut): order() not find contract for symbol:pb2602.SHF


2026-02-27 10:55:47.076877 strategy(bktfut): order() not find contract for symbol:pb2602.SHF 
2026-02-27 10:55:47.077472 strategy(bktfut): order() not find contract for symbol:pb2602.SHF 
[2026-02-27 10:55:47] [info     ] backtest done, raw_perf_ds:dai.DataSource("_2ac31e9cde354bd88a58de3d065be3c4")


[2026-02-27 10:55:47] [info     ] bigtrader.v34 运行完成 [1.406s].
